In [44]:
!pip install lightGBM

In [45]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.metrics import f1_score,confusion_matrix,classification_report,average_precision_score
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from xgboost import XGBClassifier
from os.path import exists
import pickle
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [65]:
test = pd.read_csv('./test.csv')
train = pd.read_csv('./train.csv')
sub = pd.read_csv('./sample_submission.csv')

In [66]:
test.dtypes

Unnamed: 0                    int64
loan_id                       int64
source                       object
financial_institution        object
interest_rate               float64
unpaid_principal_bal          int64
loan_term                     int64
origination_date             object
first_payment_date           object
loan_to_value                 int64
number_of_borrowers         float64
debt_to_income_ratio        float64
borrower_credit_score       float64
loan_purpose                 object
insurance_percent           float64
co-borrower_credit_score    float64
insurance_type              float64
m1                            int64
m2                            int64
m3                            int64
m4                            int64
m5                            int64
m6                            int64
m7                            int64
m8                            int64
m9                            int64
m10                           int64
m11                         

## Merge

In [67]:
train['origination_date'] = pd.to_datetime(train['origination_date'])
train['org_month'],train['org_day'],train['org_year'] =train['origination_date'].dt.month,train['origination_date'].dt.day,train['origination_date'].dt.year
train['first_payment_date']=pd.to_datetime(train['first_payment_date'])
train['fst_month'],train['fst_day'],train['fst_year'] =train['first_payment_date'].dt.month,train['first_payment_date'].dt.day,train['first_payment_date'].dt.year
train.drop(['first_payment_date','origination_date'],1,inplace=True)

In [68]:
test['origination_date'] = pd.to_datetime(test['origination_date'])
test['org_month'],test['org_day'],test['org_year'] =test['origination_date'].dt.month,test['origination_date'].dt.day,test['origination_date'].dt.year
test['first_payment_date']=pd.to_datetime(test['first_payment_date'])
test['fst_month'],test['fst_day'],test['fst_year'] =test['first_payment_date'].dt.month,test['first_payment_date'].dt.day,test['first_payment_date'].dt.year
test.drop(['first_payment_date','origination_date'],1,inplace=True)

In [69]:
cat_columns = []
for col in train.select_dtypes('object').columns:
    print(col)
    cat_columns.append(col)
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

source
financial_institution
loan_purpose


In [70]:
train

,Unnamed: 0,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,...,m10,m11,m12,m13,org_month,org_day,org_year,fst_month,fst_day,fst_year
0,81041,861482495205,1,5,3.375,272000,180,72,2.0,31.0,...,0,0,0,0,1,1,2012,3,1,2012
1,51021,173139140902,1,15,4.250,371000,360,95,1.0,45.0,...,0,0,0,0,1,1,2012,3,1,2012
2,83866,481942560913,1,8,4.500,90000,360,69,1.0,39.0,...,0,0,0,0,1,1,2012,3,1,2012
3,92311,676780245612,1,18,4.500,47000,240,75,1.0,43.0,...,0,0,0,0,2,1,2012,4,1,2012
4,72274,433043514697,0,8,4.875,177000,360,80,2.0,13.0,...,0,0,0,0,1,1,2012,3,1,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92841,64877,952892082095,0,1,3.875,68000,180,57,1.0,40.0,...,0,0,0,0,2,1,2012,4,1,2012
92842,4714,405456561770,0,8,3.000,67000,180,35,2.0,12.0,...,0,0,0,0,2,1,2012,4,1,2012
92843,108500,643283061619,0,8,4.500,81000,360,75,2.0,20.0,...,0,0,0,0,2,1,2012,4,1,2012
92844,41187,450225782159,0,8,3.875,115000,180,72,2.0,44.0,...,0,0,0,0,2,1,2012,4,1,2012


In [79]:
X,y= train.drop(['loan_id','m13','Unnamed: 0'],1),train['m13']

In [80]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,test_size = 0.15 )

In [81]:
X_test

,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,...,m9,m10,m11,m12,org_month,org_day,org_year,fst_month,fst_day,fst_year
52302,0,10,3.375,278000,180,35,2.0,17.0,787.0,0,...,0,0,0,0,1,1,2012,3,1,2012
9163,0,6,3.250,175000,120,39,2.0,17.0,812.0,0,...,0,0,0,0,2,1,2012,4,1,2012
49501,1,1,4.375,111000,360,85,1.0,21.0,768.0,1,...,0,0,0,0,2,1,2012,4,1,2012
73331,0,8,3.850,387000,360,91,2.0,38.0,780.0,0,...,0,0,0,0,3,1,2012,5,1,2012
56764,1,1,4.125,381000,360,90,2.0,31.0,750.0,2,...,0,0,0,0,2,1,2012,4,1,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31558,0,8,4.000,226000,360,80,1.0,24.0,804.0,0,...,0,0,0,0,3,1,2012,5,1,2012
75610,0,5,3.420,292000,180,71,2.0,41.0,804.0,0,...,0,0,0,0,1,1,2012,3,1,2012
78458,0,8,3.250,167000,180,59,1.0,36.0,721.0,1,...,0,0,0,0,1,1,2012,3,1,2012
7314,0,8,3.750,217000,360,85,2.0,40.0,745.0,1,...,0,0,0,0,1,1,2012,3,1,2012


In [82]:
classifiers = {
    # "CatBoost":CatBoostClassifier(),
    "LightGBM":LGBMClassifier(),
    "XGBClassifier": XGBClassifier(),
    "BaggingClassifier": BaggingClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
   # "VotingClassifier":VotingClassifier(estimators=200),
    "GradientBoostingClassifier":GradientBoostingClassifier(),
    "AdaBoostClassifier":AdaBoostClassifier(),
    "HistGradientBoostingClassifier":HistGradientBoostingClassifier()
    }


for key, classifier in classifiers.items():
    print('_' * 50)
    name = key
    print(name)
    if exists(name):
        print('loading...')
        classifier = pickle.load(open(name, 'rb'))
        training_score = cross_val_score(classifier, X_train, y_train, cv=5)
        print("Classifiers: ", name, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
    else:
        classifier.fit(X_train, y_train)
        print('validating ...')
        training_score = cross_val_score(classifier, X_train, y_train, cv=5)
        print("Classifiers: ", name, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
        pickle.dump(classifier, open(name, 'wb'))
    print(classifier.score(X_train, y_train))
    print(classifier.score(X_test, y_test))
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print(classification_report(y_test, y_pred))
    print(average_precision_score(y_test, y_pred))

__________________________________________________
LightGBM
loading...
Classifiers:  LightGBM Has a training score of 100.0 % accuracy score


ValueError: Number of features of the model must match the input. Model n_features_ is 27 and input n_features is 31 

In [47]:
classifier = pickle.load(open('./XGBClassifier', 'rb'))

In [49]:
test['m13'] = classifier.predict(test.drop(['loan_id','Unnamed: 0'],1))

In [50]:
test['m13'].value_counts()

0    23150
1       62
Name: m13, dtype: int64

In [51]:
test[['Unnamed: 0','m13']].to_csv('xgb_sub.csv',index=False)

In [93]:
new = CatBoostClassifier(eval_metric='F1',random_seed=1,auto_class_weights='Balanced').fit(X_train,y_train)

Learning rate set to 0.066532
0:	learn: 0.7660720	total: 19.2ms	remaining: 19.1s
1:	learn: 0.7806567	total: 41.7ms	remaining: 20.8s
2:	learn: 0.7995187	total: 63.8ms	remaining: 21.2s
3:	learn: 0.7973901	total: 87ms	remaining: 21.7s
4:	learn: 0.8072248	total: 107ms	remaining: 21.4s
5:	learn: 0.8059183	total: 127ms	remaining: 21.1s
6:	learn: 0.8062341	total: 144ms	remaining: 20.5s
7:	learn: 0.8076006	total: 171ms	remaining: 21.2s
8:	learn: 0.8074100	total: 192ms	remaining: 21.1s
9:	learn: 0.8119554	total: 218ms	remaining: 21.6s
10:	learn: 0.8107967	total: 234ms	remaining: 21.1s
11:	learn: 0.8146101	total: 250ms	remaining: 20.6s
12:	learn: 0.8136318	total: 267ms	remaining: 20.2s
13:	learn: 0.8117577	total: 282ms	remaining: 19.9s
14:	learn: 0.8135714	total: 297ms	remaining: 19.5s
15:	learn: 0.8121139	total: 314ms	remaining: 19.3s
16:	learn: 0.8122233	total: 330ms	remaining: 19.1s
17:	learn: 0.8141358	total: 346ms	remaining: 18.9s
18:	learn: 0.8155026	total: 362ms	remaining: 18.7s
19:	learn

165:	learn: 0.8830761	total: 3.14s	remaining: 15.8s
166:	learn: 0.8888629	total: 3.16s	remaining: 15.8s
167:	learn: 0.8890743	total: 3.19s	remaining: 15.8s
168:	learn: 0.8903515	total: 3.2s	remaining: 15.8s
169:	learn: 0.8903750	total: 3.22s	remaining: 15.7s
170:	learn: 0.8891918	total: 3.24s	remaining: 15.7s
171:	learn: 0.8909335	total: 3.26s	remaining: 15.7s
172:	learn: 0.8940729	total: 3.28s	remaining: 15.7s
173:	learn: 0.8955054	total: 3.3s	remaining: 15.6s
174:	learn: 0.8981780	total: 3.32s	remaining: 15.6s
175:	learn: 0.8942615	total: 3.34s	remaining: 15.6s
176:	learn: 0.8969524	total: 3.36s	remaining: 15.6s
177:	learn: 0.9010503	total: 3.38s	remaining: 15.6s
178:	learn: 0.9023312	total: 3.4s	remaining: 15.6s
179:	learn: 0.9023725	total: 3.42s	remaining: 15.6s
180:	learn: 0.9014166	total: 3.44s	remaining: 15.6s
181:	learn: 0.9017772	total: 3.47s	remaining: 15.6s
182:	learn: 0.9031943	total: 3.49s	remaining: 15.6s
183:	learn: 0.9087782	total: 3.51s	remaining: 15.6s
184:	learn: 0.9

329:	learn: 0.9853403	total: 6.53s	remaining: 13.3s
330:	learn: 0.9854397	total: 6.55s	remaining: 13.2s
331:	learn: 0.9854521	total: 6.58s	remaining: 13.2s
332:	learn: 0.9854583	total: 6.61s	remaining: 13.2s
333:	learn: 0.9854770	total: 6.63s	remaining: 13.2s
334:	learn: 0.9855702	total: 6.66s	remaining: 13.2s
335:	learn: 0.9858625	total: 6.68s	remaining: 13.2s
336:	learn: 0.9860367	total: 6.7s	remaining: 13.2s
337:	learn: 0.9872365	total: 6.72s	remaining: 13.2s
338:	learn: 0.9872365	total: 6.74s	remaining: 13.1s
339:	learn: 0.9873796	total: 6.76s	remaining: 13.1s
340:	learn: 0.9874232	total: 6.79s	remaining: 13.1s
341:	learn: 0.9875602	total: 6.8s	remaining: 13.1s
342:	learn: 0.9876038	total: 6.82s	remaining: 13.1s
343:	learn: 0.9888756	total: 6.84s	remaining: 13.1s
344:	learn: 0.9889130	total: 6.87s	remaining: 13s
345:	learn: 0.9889690	total: 6.89s	remaining: 13s
346:	learn: 0.9890687	total: 6.91s	remaining: 13s
347:	learn: 0.9891186	total: 6.93s	remaining: 13s
348:	learn: 0.9892183	

494:	learn: 0.9950181	total: 9.76s	remaining: 9.96s
495:	learn: 0.9951253	total: 9.78s	remaining: 9.93s
496:	learn: 0.9951568	total: 9.79s	remaining: 9.91s
497:	learn: 0.9951947	total: 9.81s	remaining: 9.89s
498:	learn: 0.9952073	total: 9.83s	remaining: 9.87s
499:	learn: 0.9952325	total: 9.85s	remaining: 9.85s
500:	learn: 0.9952389	total: 9.87s	remaining: 9.83s
501:	learn: 0.9952704	total: 9.88s	remaining: 9.8s
502:	learn: 0.9952704	total: 9.9s	remaining: 9.78s
503:	learn: 0.9952578	total: 9.92s	remaining: 9.76s
504:	learn: 0.9953019	total: 9.94s	remaining: 9.74s
505:	learn: 0.9952893	total: 9.95s	remaining: 9.72s
506:	learn: 0.9953398	total: 9.97s	remaining: 9.7s
507:	learn: 0.9953714	total: 9.99s	remaining: 9.67s
508:	learn: 0.9954282	total: 10s	remaining: 9.66s
509:	learn: 0.9954913	total: 10s	remaining: 9.64s
510:	learn: 0.9954724	total: 10s	remaining: 9.62s
511:	learn: 0.9954660	total: 10.1s	remaining: 9.6s
512:	learn: 0.9954787	total: 10.1s	remaining: 9.58s
513:	learn: 0.9954787	

657:	learn: 0.9972177	total: 12.9s	remaining: 6.71s
658:	learn: 0.9972240	total: 12.9s	remaining: 6.69s
659:	learn: 0.9972367	total: 13s	remaining: 6.67s
660:	learn: 0.9972620	total: 13s	remaining: 6.66s
661:	learn: 0.9972620	total: 13s	remaining: 6.64s
662:	learn: 0.9972747	total: 13s	remaining: 6.63s
663:	learn: 0.9972937	total: 13.1s	remaining: 6.61s
664:	learn: 0.9972937	total: 13.1s	remaining: 6.6s
665:	learn: 0.9973000	total: 13.1s	remaining: 6.58s
666:	learn: 0.9973064	total: 13.2s	remaining: 6.57s
667:	learn: 0.9973000	total: 13.2s	remaining: 6.55s
668:	learn: 0.9973317	total: 13.2s	remaining: 6.53s
669:	learn: 0.9973190	total: 13.2s	remaining: 6.52s
670:	learn: 0.9973380	total: 13.3s	remaining: 6.5s
671:	learn: 0.9973380	total: 13.3s	remaining: 6.48s
672:	learn: 0.9973444	total: 13.3s	remaining: 6.47s
673:	learn: 0.9973761	total: 13.3s	remaining: 6.45s
674:	learn: 0.9973571	total: 13.4s	remaining: 6.43s
675:	learn: 0.9973951	total: 13.4s	remaining: 6.41s
676:	learn: 0.9974331	

818:	learn: 0.9982132	total: 17s	remaining: 3.77s
819:	learn: 0.9982196	total: 17.1s	remaining: 3.75s
820:	learn: 0.9982322	total: 17.1s	remaining: 3.73s
821:	learn: 0.9982386	total: 17.1s	remaining: 3.71s
822:	learn: 0.9982322	total: 17.2s	remaining: 3.69s
823:	learn: 0.9982322	total: 17.2s	remaining: 3.67s
824:	learn: 0.9982322	total: 17.2s	remaining: 3.65s
825:	learn: 0.9982449	total: 17.3s	remaining: 3.63s
826:	learn: 0.9982386	total: 17.3s	remaining: 3.62s
827:	learn: 0.9982449	total: 17.3s	remaining: 3.6s
828:	learn: 0.9982640	total: 17.3s	remaining: 3.58s
829:	learn: 0.9982513	total: 17.4s	remaining: 3.56s
830:	learn: 0.9982513	total: 17.4s	remaining: 3.54s
831:	learn: 0.9982386	total: 17.4s	remaining: 3.52s
832:	learn: 0.9982513	total: 17.5s	remaining: 3.5s
833:	learn: 0.9982640	total: 17.5s	remaining: 3.48s
834:	learn: 0.9982640	total: 17.5s	remaining: 3.46s
835:	learn: 0.9982957	total: 17.5s	remaining: 3.44s
836:	learn: 0.9982830	total: 17.6s	remaining: 3.42s
837:	learn: 0.99

977:	learn: 0.9986768	total: 21.2s	remaining: 477ms
978:	learn: 0.9986704	total: 21.2s	remaining: 455ms
979:	learn: 0.9986704	total: 21.2s	remaining: 434ms
980:	learn: 0.9986704	total: 21.3s	remaining: 412ms
981:	learn: 0.9986641	total: 21.3s	remaining: 391ms
982:	learn: 0.9986704	total: 21.3s	remaining: 369ms
983:	learn: 0.9986641	total: 21.4s	remaining: 347ms
984:	learn: 0.9986704	total: 21.4s	remaining: 326ms
985:	learn: 0.9986641	total: 21.4s	remaining: 304ms
986:	learn: 0.9986704	total: 21.4s	remaining: 282ms
987:	learn: 0.9986831	total: 21.5s	remaining: 261ms
988:	learn: 0.9986895	total: 21.5s	remaining: 239ms
989:	learn: 0.9986895	total: 21.5s	remaining: 217ms
990:	learn: 0.9986895	total: 21.5s	remaining: 196ms
991:	learn: 0.9986831	total: 21.6s	remaining: 174ms
992:	learn: 0.9986895	total: 21.6s	remaining: 152ms
993:	learn: 0.9986831	total: 21.6s	remaining: 131ms
994:	learn: 0.9986958	total: 21.6s	remaining: 109ms
995:	learn: 0.9986958	total: 21.7s	remaining: 87ms
996:	learn: 0

In [94]:
print(new.score(X_train, y_train))
print(new.score(X_test, y_test))
y_pred = new.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
print(average_precision_score(y_test, y_pred))

0.9974657560283329
0.9941839592159115
[[13800    49]
 [   32    46]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13849
           1       0.48      0.59      0.53        78

    accuracy                           0.99     13927
   macro avg       0.74      0.79      0.76     13927
weighted avg       0.99      0.99      0.99     13927

0.2878577491056848
